# Day 09. Exercise 03
# Ensembles

## 0. Imports

In [66]:
import pandas as pd
import joblib

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, BaggingClassifier, StackingClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score

## 1. Preprocessing

1. Create the same dataframe as in the previous exercise.
2. Using `train_test_split` with parameters `test_size=0.2`, `random_state=21` get `X_train`, `y_train`, `X_test`, `y_test` and then get `X_train`, `y_train`, `X_valid`, `y_valid` from the previous `X_train`, `y_train`. Use the additional parameter `stratify`.

In [33]:
df = pd.read_csv('../data/day-of-week-not-scaled.csv')
weeks = pd.read_csv('../data/dayofweek.csv')
df["weeks"] = weeks['dayofweek']

In [34]:
X = df.drop("weeks",axis=1)
y = df["weeks"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)

In [35]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=21, stratify=y_train)

## 2. Individual classifiers

1. Train SVM, decision tree and random forest again with the best parameters that you got from the 01 exercise with `random_state=21` for all of them.
2. Evaluate `accuracy`, `precision`, and `recall` for them on the validation set.
3. The result of each cell of the section should look like this:

```
accuracy is 0.87778
precision is 0.88162
recall is 0.87778
```

In [36]:
svm = SVC(C=10, class_weight=None, gamma='auto', kernel='rbf', random_state=21, probability=True)
svm.fit(X_train, y_train)
y_pred = svm.predict(X_valid)

In [37]:
accuracy = accuracy_score(y_valid, y_pred)
precision = precision_score(y_valid, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_valid, y_pred, average='weighted', zero_division=0)

In [38]:
print(f'accuracy: {accuracy:.4f}')
print(f'precision: {precision:.4f}')
print(f'recall: {recall:.4f}')

accuracy: 0.8778
precision: 0.8816
recall: 0.8778


In [39]:
tree = DecisionTreeClassifier(class_weight='balanced', criterion='entropy', max_depth=25, random_state=21)
tree.fit(X_train, y_train)
y_pred = tree.predict(X_valid)

In [40]:
accuracy = accuracy_score(y_valid, y_pred)
precision = precision_score(y_valid, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_valid, y_pred, average='weighted', zero_division=0)

In [41]:
print(f'accuracy: {accuracy:.4f}')
print(f'precision: {precision:.4f}')
print(f'recall: {recall:.4f}')

accuracy: 0.8519
precision: 0.8548
recall: 0.8519


In [42]:
forest = RandomForestClassifier(class_weight='balanced', criterion='gini', max_depth=22, n_estimators=100, random_state=21)
forest.fit(X_train, y_train)
y_pred = forest.predict(X_valid)


In [43]:
accuracy = accuracy_score(y_valid, y_pred)
precision = precision_score(y_valid, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_valid, y_pred, average='weighted', zero_division=0)

In [44]:
print(f'accuracy: {accuracy:.4f}')
print(f'precision: {precision:.4f}')
print(f'recall: {recall:.4f}')

accuracy: 0.9000
precision: 0.9021
recall: 0.9000


## 3. Voting classifiers

1. Using `VotingClassifier` and the three models that you have just trained, calculate the `accuracy`, `precision`, and `recall` on the validation set.
2. Play with the other parameteres.
3. Calculate the `accuracy`, `precision` and `recall` on the test set for the model with the best weights in terms of accuracy (if there are several of them with equal values, choose the one with the higher precision).

In [45]:
voting_class = VotingClassifier(estimators=[('svm', svm), ('dt', tree), ('rf', forest)], voting='soft')
voting_class.fit(X_train, y_train)
y_pred= voting_class.predict(X_valid)
y_pred_test = voting_class.predict(X_test)

In [46]:
accuracy = accuracy_score(y_valid, y_pred)
precision = precision_score(y_valid, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_valid, y_pred, average='weighted', zero_division=0)

In [47]:
print(f'accuracy: {accuracy:.4f}')
print(f'precision: {precision:.4f}')
print(f'recall: {recall:.4f}')

accuracy: 0.8852
precision: 0.8871
recall: 0.8852


In [48]:
accuracy_test = accuracy_score(y_test, y_pred_test)
precision_test = precision_score(y_test, y_pred_test, average='weighted', zero_division=0)
recall_test = recall_score(y_test, y_pred_test, average='weighted', zero_division=0)

In [49]:
print(f'accuracy: {accuracy:.4f}')
print(f'precision: {precision:.4f}')
print(f'recall: {recall:.4f}')

accuracy: 0.8852
precision: 0.8871
recall: 0.8852


## 4. Bagging classifiers

1. Using `BaggingClassifier` and `SVM` with the best parameters create an ensemble, try different values of the `n_estimators`, use `random_state=21`.
2. Play with the other parameters.
3. Calculate the `accuracy`, `precision`, and `recall` for the model with the best parameters (in terms of accuracy) on the test set (if there are several of them with equal values, choose the one with the higher precision)

In [53]:
bagging_class= BaggingClassifier(svm, n_estimators=50, random_state=21)
bagging_class.fit(X_train, y_train)
y_pred_valid = bagging_class.predict(X_valid)
y_pred_test = bagging_class.predict(X_test)

In [51]:
accuracy_valid = accuracy_score(y_valid, y_pred_valid)
precision_valid = precision_score(y_valid, y_pred_valid, average='weighted', zero_division=0)
recall_valid = recall_score(y_valid, y_pred_valid, average='weighted', zero_division=0)

In [52]:
print(f'accuracy: {accuracy:.4f}')
print(f'precision: {precision:.4f}')
print(f'recall: {recall:.4f}')

accuracy: 0.8852
precision: 0.8871
recall: 0.8852


In [54]:
accuracy_test = accuracy_score(y_test, y_pred_test)
precision_test = precision_score(y_test, y_pred_test, average='weighted', zero_division=0)
recall_test = recall_score(y_test, y_pred_test, average='weighted', zero_division=0)

In [55]:
print(f'accuracy: {accuracy_test:.4f}')
print(f'precision: {precision_test:.4f}')
print(f'recall: {recall_test:.4f}')

accuracy: 0.8846
precision: 0.8894
recall: 0.8846


## 5. Stacking classifiers

1. To achieve reproducibility in this case you will have to create an object of cross-validation generator: `StratifiedKFold(n_splits=n, shuffle=True, random_state=21)`, where `n` you will try to optimize (the details are below).
2. Using `StackingClassifier` and the three models that you have recently trained, calculate the `accuracy`, `precision` and `recall` on the validation set, try different values of `n_splits` `[2, 3, 4, 5, 6, 7]` in the cross-validation generator and parameter `passthrough` in the classifier itself,
3. Calculate the `accuracy`, `precision`, and `recall` for the model with the best parameters (in terms of accuracy) on the test set (if there are several of them with equal values, choose the one with the higher precision). Use `final_estimator=LogisticRegression(solver='liblinear')`.

In [59]:

skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=21)

stacking_class = StackingClassifier(
        estimators=[('svm', svm), ('dt', tree), ('rf', forest)],
        final_estimator=LogisticRegression(solver='liblinear'),
        passthrough=True, 
        cv=skf
)
stacking_class.fit(X_train, y_train)
y_pred = stacking_class.predict(X_test)

In [ ]:
accuracy_test = accuracy_score(y_test, y_pred)
precision_test = precision_score(y_test, y_pred, average='weighted', zero_division=0)
recall_test = recall_score(y_test, y_pred, average='weighted', zero_division=0)

In [58]:
print(f'accuracy: {accuracy_test:.4f}')
print(f'precision: {precision_test:.4f}')
print(f'recall: {recall_test:.4f}')

accuracy: 0.9201
precision: 0.9209
recall: 0.9201


## 6. Predictions

1. Choose the best model in terms of accuracy (if there are several of them with equal values, choose the one with the higher precision).
2. Analyze: for which weekday your model makes the most errors (in % of the total number of samples of that class in your full dataset), for which labname and for which users.
3. Save the model.

In [ ]:
best_model = stacking_class

error_analysis = {
    'weekday_errors': {},
    'labname_errors': {},
    'user_errors': {}
}

df['predictions'] = best_model.predict(X)
df['errors'] = df['predictions'] != df['weeks']

In [62]:
for weekday in df['weeks'].unique():
    total_samples = df[df['weeks'] == weekday].shape[0]
    error_samples = df[(df['weeks'] == weekday) & (df['errors'])].shape[0]
    error_rate = (error_samples / total_samples) * 100 if total_samples > 0 else 0
    error_analysis['weekday_errors'][weekday] = error_rate
error_analysis['weekday_errors']

{4: 2.8846153846153846,
 5: 4.059040590405904,
 6: 2.247191011235955,
 0: 8.088235294117647,
 1: 2.9197080291970803,
 2: 4.026845637583892,
 3: 2.0202020202020203}

In [63]:
for labname in df.columns[df.columns.str.startswith('labname_')]:
    total_samples = df[df[labname] == 1].shape[0]
    error_samples = df[(df[labname] == 1) & (df['errors'])].shape[0]
    error_rate = (error_samples / total_samples) * 100 if total_samples > 0 else 0
    error_analysis['labname_errors'][labname] = error_rate
error_analysis['labname_errors']

{'labname_code_rvw': 6.097560975609756,
 'labname_lab02': 0.0,
 'labname_lab03': 100.0,
 'labname_lab03s': 0.0,
 'labname_lab05s': 5.555555555555555,
 'labname_laba04': 5.617977528089887,
 'labname_laba04s': 7.6923076923076925,
 'labname_laba05': 0.9009009009009009,
 'labname_laba06': 6.25,
 'labname_laba06s': 9.836065573770492,
 'labname_project1': 1.8927444794952681}

In [64]:
for user_col in df.columns[df.columns.str.startswith('uid_user_')]:
    total_samples = df[df[user_col] == 1].shape[0]
    error_samples = df[(df[user_col] == 1) & (df['errors'])].shape[0]
    error_rate = (error_samples / total_samples) * 100 if total_samples > 0 else 0
    error_analysis['user_errors'][user_col] = error_rate
error_analysis['user_errors']

{'uid_user_0': 0.0,
 'uid_user_1': 2.1739130434782608,
 'uid_user_10': 1.4084507042253522,
 'uid_user_11': 0.0,
 'uid_user_12': 0.0,
 'uid_user_13': 1.6666666666666667,
 'uid_user_14': 5.303030303030303,
 'uid_user_15': 0.0,
 'uid_user_16': 6.25,
 'uid_user_17': 5.88235294117647,
 'uid_user_18': 5.714285714285714,
 'uid_user_19': 4.395604395604396,
 'uid_user_2': 4.958677685950414,
 'uid_user_20': 0.0,
 'uid_user_21': 4.545454545454546,
 'uid_user_22': 28.57142857142857,
 'uid_user_23': 25.0,
 'uid_user_24': 3.571428571428571,
 'uid_user_25': 2.5,
 'uid_user_26': 1.1111111111111112,
 'uid_user_27': 4.3478260869565215,
 'uid_user_28': 0.0,
 'uid_user_29': 6.25,
 'uid_user_3': 2.8169014084507045,
 'uid_user_30': 2.564102564102564,
 'uid_user_31': 4.0,
 'uid_user_4': 1.5957446808510638,
 'uid_user_6': 16.666666666666664,
 'uid_user_7': 0.0,
 'uid_user_8': 4.25531914893617}

In [65]:
joblib.dump(best_model, 'my_model_ex03.pkl')

['my_model_ex03.pkl']